## Homework 1
#### Aly Kapasi
#### ECS171 SS1-21
#### July 6, 2021

In [39]:
from math import exp
import numpy as np
import pandas as pd
import random
from random import random
from random import seed
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

seed(1)
dry_beans = pd.read_csv('Dry_Beans_Dataset.csv')

def min_max_normalization(X) :
    X = (X - X.min()) / (X.max() - X.min())
    return X

def grid_search(clf, parameters, X, y):
    gridSearch = GridSearchCV(clf, param_grid=parameters, n_jobs=-1, cv=5) 
    gridSearch.fit(X,y)
    params = gridSearch.best_params_
    score = gridSearch.best_score_
    return gridSearch, params, score

### Part 1
Design a 4-layer artificial neural network, specifically a feed-forward multi-layer perceptron (using the sigmoid activation function), to classify the type of 'Dry Bean' given the other attributes in the data set, similar to the one mentioned in the paper above. For this, split the data into training and testing set by 90:10 and use the training set for training the model and the test set to evaluate the model performance. Please note that this is a multi-class classification problem so select the right number of nodes accordingly for the output layer.

Consider the following hyperparameters :

Number of nodes in each hidden layer should be (12, 3).
Learning rate should be 0.3
Number of epochs should be 500
The sigmoid function should be used as the activation function in each layer
Stochastic Gradient Descent should be used to minimize the error rate
Once the model has been trained, test the model and obtain the following :

The confusion matrix for each class, specifying the true positive, true negative, false positive, and false negative cases for each category in the class

In [40]:
x = dry_beans.drop("Class",1).to_numpy()
y = dry_beans.iloc[:,16]

for i in range(x.shape[1]):
    x[:,i] = min_max_normalization(x[:,i])
#print(x)

label_encoder = LabelEncoder()
label_encoded =  label_encoder.fit_transform(y) 
#print(label_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
label_encoded = label_encoded.reshape(len(label_encoded), 1) 
y = onehot_encoder.fit_transform(label_encoded)
#print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)

sc_x = MinMaxScaler()
x_scaled = sc_x.fit_transform(x)
x_trainscaled=sc_x.fit_transform(x_train)
x_testscaled=sc_x.fit_transform(x_test)

clf = MLPClassifier(hidden_layer_sizes=(12,3),activation="logistic",solver="sgd",learning_rate_init=0.3,max_iter=500,random_state=1)
clf.fit(x_trainscaled, y_train)
y_pred = clf.predict(x_testscaled) 
model1_score = clf.score(x_testscaled, y_test)
model1_mse = mean_squared_error(y_test, y_pred)

print('The 4 Layer Feed-Forward Artificial Neural Network has an accuracy rate of', model1_score)
print('The 4 Layer Feed-Forward Artificial Neural Network has a mean squared error of', model1_mse)

# confusion matrix
cm = multilabel_confusion_matrix(y_test, y_pred)
tp = cm[:,1,1]
tn = cm[:,0,0]
fp = cm[:,0,1]
fn = cm[:,1,0]

print("true positive:  ", tp)
print("true negative:  ", tn)
print("false positive: ", fp)
print("false negative: ", fn)

print(classification_report(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


The 4 Layer Feed-Forward Artificial Neural Network has an accuracy rate of 0.828928046989721
The 4 Layer Feed-Forward Artificial Neural Network has a mean squared error of 0.03985735263268302
true positive [145  45 132 236 179 192 216]
true negative [1166 1317 1196 1012 1161 1147 1010]
false positive [48  0 16  2 13  9 75]
false negative [  3   0  18 112   9  14  61]
              precision    recall  f1-score   support

           0       0.75      0.98      0.85       148
           1       1.00      1.00      1.00        45
           2       0.89      0.88      0.89       150
           3       0.99      0.68      0.81       348
           4       0.93      0.95      0.94       188
           5       0.96      0.93      0.94       206
           6       0.74      0.78      0.76       277

   micro avg       0.88      0.84      0.86      1362
   macro avg       0.89      0.89      0.88      1362
weighted avg       0.89      0.84      0.86      1362
 samples avg       0.83      0.84 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


### Part 2
In order to avoid using biased models, use 10-fold cross validation to generalize the model based on the given data set. Obtain the accuracy and MSE and during each iteration of the cross validation, and find the average of both metrics.

In [41]:
cv_score = cross_val_score(clf,x_scaled,y, cv=10)
print(cv_score)
print('Accuracy: %.3f (%.3f)' % (cv_score.mean(), cv_score.std()))
print('Mean squared error: %.3f' % (np.mean(np.abs(cv_score))))

[0.89133627 0.88831741 0.91109478 0.88831741 0.90374724 0.88611315
 0.90742101 0.91182954 0.89566495 0.86554004]
Accuracy: 0.895 (0.014)
Mean squared error: 0.895


### Part 3
Use the grid search methodology to find the optimal number of nodes required in each hidden layer, as well as the optimal learning rate and the number of epochs, such that the accuracy of the model is maximum for the given data set. Also, find the accuracy of the optimized model.

In [42]:
max_iter = [50 * n for n in range(1,11)]
learning_rate_init = [0.01 * n for n in range(1,11)]
parameter_space = {
    'max_iter' : max_iter,
    'learning_rate_init' : learning_rate_init
}

clf1, params, score = grid_search(clf, parameter_space, x, y)

print('Best parameters found :\n', params)
print('Best score :\n', score)

means = clf1.cv_results_['mean_test_score']
stds = clf1.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf1.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

means = means.reshape((10, 10))
means = pd.DataFrame(means, index = learning_rate_init, columns = max_iter)
print(means.head())

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,10))
 
sns.heatmap(means, annot = True)

KeyboardInterrupt: 